In [1]:
#Import packages
import pandas as pd
import numpy as np
import re
import warnings
import math
warnings.filterwarnings('ignore')

## GUIDE-IT

In [2]:
#IMPORT FILES NEEDED

demog = pd.read_csv("../Data Validation/GUIDE-IT/Raw Data/demog_ads.csv", sep=",", index_col='deidnum').sort_index()
#need to update all these to upload correct files with data needed
hfchdxps16 = pd.read_csv("../Data Validation/ARIC/Raw Data/cohort/hfchdxps16.csv", sep=",", index_col='ID_S0').sort_index()
hfchfaps16 = pd.read_csv("../Data Validation/ARIC/Raw Data/cohort/hfchfaps16.csv", sep=",", index_col='ID_S0').sort_index()
hfcchips16 = pd.read_csv("../Data Validation/ARIC/Raw Data/cohort/hfcchips16.csv", sep=",", index_col='ID_S0').sort_index()
hfccfdps16 = pd.read_csv("../Data Validation/ARIC/Raw Data/cohort/hfccfdps16.csv", sep=",", index_col='ID_S0').sort_index()
hfccelps16 = pd.read_csv("../Data Validation/ARIC/Raw Data/cohort/hfccelps16.csv", sep=",", index_col='ID_S0').sort_index()


In [3]:
#DEMOGRAPHIC

cohort = pd.DataFrame()
cohort['ID'] = demog.index
cohort = cohort.set_index('ID')

#continue on from here

cohort['Age'] = hfcoccps16['AGE'].tolist()

lst = []
for idx in hfcoccps16.index:
    pt = hfcoccps16.loc[idx]
    if pt['SEX'] == 'M': 
        lst.append(1) #male is 1
    else:
        lst.append(2) #female 2
cohort['Gender'] = lst


lst = []
for idx in hfcchips16.index:
    pt = hfcchips16.loc[idx]
    if pt['RACEGRP'] == 'W':#white is 1, all other races are 2
        lst.append(1)
    else:
        lst.append(2)
cohort['Race'] = lst

#weight, convert to all be Kg
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA20A1'] == 'L': #convert lbs to Kg
        lst.append(pt['HFAA20A'] * 0.45359237)
    else:
        lst.append(pt['HFAA20A'])
        
cohort['Wt_B'] = lst #B = at Baseline

lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA20B1'] == 'L': #convert lbs to Kg
        lst.append(pt['HFAA20B'] * 0.45359237)
    else:
        lst.append(pt['HFAA20B'])

cohort['Wt_D'] = lst

cohort['BMI_D'] = hfcoccps16['BMI'].tolist()
cohort['EjF'] = hfcoccps16['LVEF_CUR'].tolist()

#NYHA
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA24D'] == 'N': 
        lst.append(0)
    else:
        lst.append(pt['HFAA24D']) #number from 1 to 4
cohort['NYHA'] = lst

allData['InitialHospDays'] = patient['HOSPDAY'] #how many days in first hospitalization
cohort['TotalHospDays'] = hfcoccps16['LOS'].tolist() #total length of stay in hospital, cumulative

cohort.head(5)

,Age,Gender,Race,Wt_B,Wt_D,BMI_D,EjF,NYHA,TotalHospDays
ID,,,,,,,,,
S105655,76,1,1,NaN,NaN,NaN,NaN,NaN,11.0
S105737,76,2,1,67.7,63.3,28.185923,40.0,NaN,3.0
S106083,66,2,1,86.0,88.5,36.865164,NaN,0,10.0
S106142,77,2,1,79.4,66.8,22.391913,60.0,0,11.0
S106248,66,2,1,91.0,93.3,37.621043,NaN,NaN,4.0


In [ ]:
#conditions, Ischemic
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA6A'] == 'Y' or pt['HFAA6A'] == 'Y' or pt['HFAA11K'] == 'Y' or  pt['HFAA16J']== 'Y' or  pt['HFAA76']== 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['ISCH'] = lst

#Nonischemic
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA6B'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['NonISCH'] = lst

#AF
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11B1'] == 'Y' or pt['HFAA16K'] == 'Y' or pt['HFAA26C'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['AF'] = lst

#AlchE
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA9B'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['AlchE'] = lst

#Angina
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11A'] == 'Y' or pt['HFAA16J'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['ANGP'] = lst

#ARRH
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11B1'] == 'Y' or pt['HFAA11B2'] == 'Y' or pt['HFAA11B3'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['ARRH'] = lst

#CARREST
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11D'] == 'Y' or pt['HFAA74'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['CARREST'] = lst

#COPD
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA10B'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['COPD'] = lst

#Depr
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA14B'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['DEPR'] = lst

#Diab
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA12A'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['DIAB'] = lst

#HEPT
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA22D'] == 'Y' or pt['HFAA22C'] == 'Y' or pt['HFAA12C'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['HEPT'] = lst

#HTN
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11J'] == 'Y' or pt['HFAA11L'] == 'Y' or pt['HFAA29D10'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['HTN'] = lst

#Malig
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA9E'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['MALIG'] = lst

#Malig
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA13A'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['RENAL'] = lst

#Smok
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA9H'] == 'Y' or pt['HFAA9G'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['SMOKING'] = lst

#Stroke TIA
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA14A'] == 'Y' or pt['HFAA75'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['StrokeTIA'] = lst

#VAHD
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11M'] == 'Y' or pt['HFAA28L'] == 'Y' or pt['HFAA28I'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['VAHD'] = lst

#VF
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11B3'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['VF'] = lst

#VHD
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11O'] == 'Y' or pt['HFAA29D4'] == 'Y' or pt['HFAA29D5'] == 'Y' or pt['HFAA29D6'] == 'Y' or pt['HFAA29D7'] == 'Y' or pt['HFAA29D8'] == 'Y' or pt['HFAA32B4'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['VHD'] = lst

#VT
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11B3'] == 'Y' or pt['HFAA26F'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['VT'] = lst

#CABG
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11E1'] == 'Y' or pt['HFAA51'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['CABG'] = lst

#HTRANS
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA56'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['HTRANS'] = lst

#ICD
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11E5'] == 'Y' or pt['HFAA11I'] == 'Y' or pt['HFAA47'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['ICD'] = lst

#Pace
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11E4'] == 'Y' or pt['HFAA49'] == 'Y' or pt['HFAA50'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['PACE'] = lst

#PTCI
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA11E2'] == 'Y' or pt['HFAA52'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['PTCI'] = lst


cohort.head(5)

In [ ]:
# Labs
cohort['BUN_D'] = hfchfaps16['HFAA45B'].tolist()
cohort['CRT_B'] = hfchfaps16['HFAA44A1'].tolist()

#CRT
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if not np.isnan(pt['HFAA44B1']):
        lst.append(pt['HFAA44B1'])
    else:
        lst.append(pt['HFAA44B'])

cohort['CRT_D'] = lst
cohort['HEC_D'] = hfchfaps16['HFAA38B'].tolist()
cohort['HEM_D'] = hfchfaps16['HFAA37B'].tolist()
cohort['SOD_D'] = hfchfaps16['HFAA43B'].tolist()

#check for other labs here

#Check for exercise

#Meds
lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA59'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['ACE_B'] = lst

lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA59A'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['ACE_D'] = lst

lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA65'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['BET_B'] = lst

lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA65A'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['BET_D'] = lst

lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA71'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['NIT_B'] = lst

lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA71A'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['NIT_D'] = lst

lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA68'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['DIUR_B'] = lst

lst = []
for idx in hfchfaps16.index:
    pt = hfchfaps16.loc[idx]
    if pt['HFAA68A'] == 'Y': 
        lst.append(1)
    else:
        lst.append(0)
cohort['DIUR_D'] = lst

cohort

In [ ]:
cohort['HRTRT'] = hfchfaps16['HFAA18A'].tolist()
cohort['BPSYS_B'] = hfchfaps16['HFAA17A'].tolist()
cohort['BPDIAS_B'] = hfchfaps16['HFAA17B'].tolist()
cohort['BPSYS_D'] = hfchfaps16['HFAA17C'].tolist()
cohort['BPDIAS_D'] = hfchfaps16['HFAA17D'].tolist()

cohort['RAP'] = hfchfaps16['HFAA31B1'].tolist()
cohort['PAS'] = hfchfaps16['HFAA31B2'].tolist()
cohort['PCWP'] = hfchfaps16['HFAA31B3'].tolist()
cohort['CO'] = hfchfaps16['HFAA31B4'].tolist()
cohort['CI'] = hfchfaps16['HFAA31B5'].tolist()


#Calculate all composite hemo for you --> change data file name to cohort for all these
idx = sorted(set(hemoComposite.index))

#MAP
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = t['BPSYS_B'] + ((2 * t['BPDIAS_B']) / 3)
    f = t['BPSYS_D'] + ((2 * t['BPDIAS_D']) / 3)
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['MAP_B'] = base
hemoComposite['MAP_D'] = final
hemoComposite['MAP_Chg'] = chg

#MPAP
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = t['PAS_B'] + ((2 * t['PAD_B']) / 3)
    f = t['PAS_D'] + ((2 * t['PAD_D']) / 3)
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['MPAP_B'] = base
hemoComposite['MPAP_D'] = final
hemoComposite['MPAP_Chg'] = chg

#CPI
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = (t['CI_B'] * t['MAP_B']) / 451
    f = (t['CI_D'] * t['MAP_D']) / 451
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['CPI_B'] = base
hemoComposite['CPI_D'] = final
hemoComposite['CPI_Chg'] = chg
    
#PP
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = t['BPSYS_B'] - t['BPDIAS_B']
    f = t['BPSYS_D'] - t['BPDIAS_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['PP_B'] = base
hemoComposite['PP_D'] = final
hemoComposite['PP_Chg'] = chg

#PPP
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = t['PP_B'] / t['BPSYS_B']
    f = t['PP_D'] / t['BPSYS_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['PPP_B'] = base
hemoComposite['PPP_D'] = final
hemoComposite['PPP_Chg'] = chg
    
#PAPP
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = (t['PAS_B']  - t['PAD_B']) / t['PAS_B']
    f = (t['PAS_D']  - t['PAD_D']) / t['PAS_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['PAPP_B'] = base
hemoComposite['PAPP_D'] = final
hemoComposite['PAPP_Chg'] = chg

#SVR
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = 80 * (t['MAP_B']  - t['RAP_B']) / t['CO_B']
    f = 80 * (t['MAP_D']  - t['RAP_D']) / t['CO_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['SVR_B'] = base
hemoComposite['SVR_D'] = final
hemoComposite['SVR_Chg'] = chg

#RAT
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = t['RAP_B'] / t['PCWP_B']
    f = t['RAP_D'] / t['PCWP_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['RAT_B'] = base
hemoComposite['RAT_D'] = final
hemoComposite['RAT_Chg'] = chg

#PPRatio
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = t['PP_B'] / t['HRTRT_B']
    f = t['PP_D'] / t['HRTRT_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['PPRatio_B'] = base
hemoComposite['PPRatio_D'] = final
hemoComposite['PPRatio_Chg'] = chg

#PAPi
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = (t['PAS_B'] - t['PAD_B']) / t['RAP_B']
    f = (t['PAS_D'] - t['PAD_D']) / t['RAP_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['PAPi_B'] = base
hemoComposite['PAPi_D'] = final
hemoComposite['PAPi_Chg'] = chg

#SAPi
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = (t['BPSYS_B'] - t['BPDIAS_B']) / t['PCWP_B']
    f = (t['BPSYS_D'] - t['BPDIAS_D']) / t['PCWP_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['SAPi_B'] = base
hemoComposite['SAPi_D'] = final
hemoComposite['SAPi_Chg'] = chg

#CPP
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = t['BPDIAS_B'] - t['PCWP_B']
    f = t['BPDIAS_B'] - t['PCWP_B']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['CPP_B'] = base
hemoComposite['CPP_D'] = final
hemoComposite['CPP_Chg'] = chg

#PRAPRat
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = t['PP_B'] / t['RAP_B']
    f = t['PP_D'] / t['RAP_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['PRAPRat_B'] = base
hemoComposite['PRAPRat_D'] = final
hemoComposite['PRAPRat_Chg'] = chg

hemoComposite.head(5)

In [ ]:
#Make separate datasets --> shouldn't need to change

#HEMO
varz = ['Age', 'Gender', 'Race', 'EjF', 'RAP', 'PAS', 'PAD', 'PAMN', 'PCWP', 'CO', 
                            'CI', 'MIXED', 'BPSYS', 'BPDIAS', 'HRTRT', 'MAP','MPAP', 'CPI',
                            'PP', 'PPP', 'PAPP', 'SVR', 'RAT', 'PPRatio', 'PAPi', 'SAPi', 'CPP', 'PRAPRat']

giantList = []
for i in range(len(cohort)):
    df = cohort.iloc[i]
    row0 = []
    row1 = []
    row0.append(df.name)
    row1.append(df.name)
    for col in varz:
        if col in cohort.columns: #value for both
            row0.append(df[col])
            row1.append(df[col])
        elif col + "_B" in cohort.columns and col + "_D" in cohort.columns:
            row0.append(df[col+"_B"])
            row1.append(df[col+"_D"])
        elif col + "_B" in cohort.columns and not col + "_D" in cohort.columns:
            row0.append(df[col+"_B"])
            row1.append(0)
        elif not col + "_B" in cohort.columns and col + "_D" in cohort.columns:
            row0.append(0)
            row1.append(df[col+"_D"])
        else:
            row0.append(0)
            row1.append(0)
    
    giantList.append(row0)
    giantList.append(row1)

varz.insert(0, 'ID')
hemoDF = pd.DataFrame(giantList, columns=varz).set_index('ID', drop=True)
    
hemoDF

In [ ]:
#Get only patients with hemo
hemoTest = hemoDF[['RAP', 'PAS', 'PCWP', 'CO', 'CI']]
hemoComposite = hemoDF.loc[~(np.isnan(hemoTest)).all(axis=1)]
print(hemoComposite.shape)
hemoComposite.head(5)


In [ ]:
#save to file
hemoComposite.to_csv("../Data Validation/ARIC/Original DataFrames/HemoAricCohort.csv")

In [ ]:
#MAke Hemo Labels
labels = pd.DataFrame()
labels['ID'] = hemoComposite.index
labels = labels.set_index('ID')

#Make Labels
labels = pd.DataFrame()
labels['ID'] = hfcoccps16.index
labels = labels.set_index('ID')

dthList =  []
rhspList = []
idx = sorted(set(patient.index))
for i in idx:
    t = patient.loc[i]
    #Death outcome
    dth = t["DEATH"] + t['LVADANY'] + t['TRANSP'] #Death is death, LVAD or Transplant 
    if dth >= 1:
        dthList.append(1)
    else:
        dthList.append(0)
        
    #Rehosp outcome
    rhsp = t['REHOSP'] #Any rehospitalization
    if rhsp >= 1:
        rhspList.append(1)
    else:
        rhspList.append(0)
        

#make readmit labels --> rehospitalization in <= 30 days
reAdList = []
idx = sorted(set(patient.index))
for i in idx:
    t = patient.loc[i]
    adt = t['T2REHSP']  #days to first rehosp
    if adt < 30:
        reAdList.append(1)
    else:
        reAdList.append(0)


In [ ]:
labels.to_csv("../Data Validation/ARIC/Original DataFrames/HemoLabelsAricCohort.csv")

In [ ]:
#Make separate datasets

#All Data - conditions
varz = ['Age', 'Gender', 'Race', 'Wt', 'BMI', 
       'InitialHospDays', 'TotalHospDays', 'NYHA', 'MLHFS', 'AF',
       'AlchE', 'ANGP', 'ARRH', 'CARREST', 'CVD', 'COPD', 'DEPR', 'DIAB',
       'GOUT', 'HEPT', 'HTN', 'MALIG', 'RENAL', 'SMOKING', 'STERD',
       'StrokeTIA', 'VAHD', 'VF', 'VHD', 'VT', 'ISCH', 'NonISCH', 'CABG',
       'HTRANS', 'ICD', 'PACE', 'PTCI', 'SixFtWlk', 'VO2','ALB', 'ALT', 'AST', 'BUN',
       'CRT', 'DIAL', 'HEC', 
       'HEM','PLA', 'POT',  'SOD',
       'TALB', 'TOTP', 'WBC', 'ACE','BET', 'NIT', 'DIUR',
       'EjF', 'BPDIAS', 'BPSYS', 'HR',
       'PV', 'MAP', 'PP', 'PPP', 'PPRatio']



giantList = []
for i in range(len(cohort)):
    df = cohort.iloc[i]
    row0 = []
    row1 = []
    row0.append(df.name)
    row1.append(df.name)
    for col in varz:
        if col in cohort.columns: #value for both
            row0.append(df[col])
            row1.append(df[col])
        elif col + "_B" in cohort.columns and col + "_D" in cohort.columns:
            row0.append(df[col+"_B"])
            row1.append(df[col+"_D"])
        elif col + "_B" in cohort.columns and not col + "_D" in cohort.columns:
            row0.append(df[col+"_B"])
            row1.append(0)
        elif not col + "_B" in cohort.columns and col + "_D" in cohort.columns:
            row0.append(0)
            row1.append(df[col+"_D"])
        else:
            row0.append(0)
            row1.append(0)
    
    giantList.append(row0)
    giantList.append(row1)

varz.insert(0, 'ID')
allData = pd.DataFrame(giantList, columns=varz).set_index('ID', drop=True)
    
allData

In [ ]:
#save to file
allData.to_csv("../Data Validation/ARIC/Original DataFrames/AllDataAricCohort.csv")

In [ ]:
#Make Labels
labels = pd.DataFrame()
labels['ID'] = hfcoccps16.index
labels = labels.set_index('ID')

dthList =  []
rhspList = []
idx = sorted(set(patient.index))
for i in idx:
    t = patient.loc[i]
    #Death outcome
    dth = t["DEATH"] + t['LVADANY'] + t['TRANSP'] #Death is death, LVAD or Transplant 
    if dth >= 1:
        dthList.append(1)
    else:
        dthList.append(0)
        
    #Rehosp outcome
    rhsp = t['REHOSP'] #Any rehospitalization
    if rhsp >= 1:
        rhspList.append(1)
    else:
        rhspList.append(0)
        

#make readmit labels --> rehospitalization in <= 30 days
reAdList = []
idx = sorted(set(patient.index))
for i in idx:
    t = patient.loc[i]
    adt = t['T2REHSP']  #days to first rehosp
    if adt < 30:
        reAdList.append(1)
    else:
        reAdList.append(0)

labels

In [ ]:
labels.to_csv("../Data Validation/ARIC/Original DataFrames/LabelsAricCohort.csv")